In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('EXPORTED CHAT FROM WHATSAPP HERE', sep = '\n', header=None)

In [ ]:
df = df.values

In [ ]:
import spacy 
nlp = spacy.load('en_core_web_sm')

In [ ]:
lines_with_tokens = []
for line in df:
    doc= nlp(line[0])    #to access the string of the array of df
    tokens=[token.text for token in doc]
    lines_with_tokens.append(tokens)

In [ ]:
i = 0
while i<len(lines_with_tokens):
  if len(lines_with_tokens[i])>5 and lines_with_tokens[i][5] == 'NAME OF OTHER USER':
    del lines_with_tokens[i]
  else:
    i +=1

In [ ]:
len(lines_with_tokens)

In [ ]:
import string
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
words_list = ['Media', 'omitted', 'pm', 'am', 'YOUR OWN NAME']
digits = '0123456789'
def message_cleaning(msg):
  msg = [char for char in msg if char not in string.punctuation and char not in digits]
  msg = "".join(msg)
  msg = [char for char in msg.split() if char not in words_list]
  msg = " ".join(msg)
 # msg = emoji(msg)
 # msg = [words for words in msg.split() if words.lower() not in stopwords.words('english')]
  return msg

In [ ]:
i = 0
while i < len(lines_with_tokens):
  lines_with_tokens[i] = " ".join(lines_with_tokens[i])
  i+=1

In [ ]:
lines_with_tokens

In [ ]:
i = 0
while i < len(lines_with_tokens):
  lines_with_tokens[i] = message_cleaning(lines_with_tokens[i])
  i+=1

In [ ]:
lines_with_tokens

In [ ]:
i = 0
while i < len(lines_with_tokens):
  lines_with_tokens[i] = lines_with_tokens[i].split()
  i+=1

In [ ]:
i = 0
while i<len(lines_with_tokens):
  for j in lines_with_tokens[i]:
    if j.isalpha() == False:
      lines_with_tokens[i].remove(j)
  i += 1 

In [ ]:
i = 0
while i < len(lines_with_tokens):
  if len(lines_with_tokens[i]) <= 1:
    del lines_with_tokens[i]
  else:
    i+=1

In [ ]:
i = 0
while i<len(lines_with_tokens):
  lines_with_tokens[i] = " ".join(lines_with_tokens[i])
  i+=1

lines_with_tokens

In [ ]:
lines_with_tokens

## Tokenization


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the words in our headlines
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines_with_tokens)
total_words = len(tokenizer.word_index) + 1 
print('Total words: ', total_words)

In [ ]:
tokenizer1 = Tokenizer()
len(tokenizer1.word_index)+1

## Creating Sequences

In [ ]:
# Convert data to sequence of tokens 
input_sequences = []
for line in lines_with_tokens:
   
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    # Create a series of sequences for each line
    for i in range(1, len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

print(tokenizer.sequences_to_texts(input_sequences[:5]))

In [ ]:
input_sequences

## Padding Sequences

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Determine max sequence length
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences with zeros at the beginning to make them all max length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[0]

In [ ]:
max_sequence_len

In [ ]:
input_sequences.shape

## Creating Predictors and Target

In [ ]:
input_df = pd.DataFrame(input_sequences)

In [ ]:
input_df

In [ ]:
input_df.to_csv('inputs.csv')

In [ ]:
## Uncomment and run cells from here on restarting

#import pandas as pd
#input_sequences = pd.read_csv('inputs.csv')

In [ ]:
input_sequences

In [ ]:
del input_sequences['Unnamed: 0']

In [ ]:
input_sequences

In [ ]:
input_sequences = input_sequences.values

In [ ]:
# Predictors are every word except the last
predictors = input_sequences[:,:-1]
# Labels are the last word
labels = input_sequences[:,-1]

In [ ]:
predictors.shape

In [ ]:
labels

In [ ]:
from tensorflow.keras import utils
total_words= 5880
labels = utils.to_categorical(labels, num_classes=total_words)

## Creating the Model

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
max_sequence_len = 152
input_len = max_sequence_len - 1 

model = Sequential()
model.add(Embedding(5880, 32, input_length=input_len))
model.add(LSTM(300, return_sequences=True))     #return_sequences=True to output a 3D array as input for the subsequent layer
model.add(Dropout(0.2))
model.add(LSTM(200))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(predictors, labels, epochs=250, verbose=1)    

In [ ]:
model.save("Model.h5")

In [ ]:
from keras.models import load_model
# load model
model = load_model("Model.h5")

## Making Predictions

In [ ]:
def predict_next_token(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    prediction = np.argmax(model.predict(token_list), axis=-1)
    return prediction

In [ ]:
prediction = predict_next_token("INPUT TEXT HERE")
tokenizer.sequences_to_texts([prediction])